In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm.notebook import tqdm

import os, sys
module_path = os.path.abspath(os.path.join('./download-tick-from-dukascopy'))
if module_path not in sys.path:
    sys.path.append(module_path)
import download_tick_from_dukascopy

path = 'data'

In [2]:
# update tick data

# for symbol, gather up all .csv, concat, and dedupe
def get_all(symbol, path):
    d = pd.DataFrame()
    for x in [x for x in os.listdir(path) if symbol in x]:
        try:
            d = pd.concat([
                d,
                pd.read_csv(path + x, header=None)
            ])
        except Exception as e:
            print(x, e)
    d.columns = ['ts', 'bid', 'ask', 'bidv', 'askv']
    d['ts'] = pd.to_datetime(d['ts'])
    d = d.set_index('ts').resample('1T').last()
    # d['mid'] = (d['bid'] + d['ask']) / 2
    # d['ch'] = d['mid'].diff()
    d = d.dropna()
    d.reset_index()['ts'].apply(lambda x: x.date()).value_counts().sort_index()#.sort_values('ts')
    return d

def start_dl(a, sym, start, end, path):
    a.main(sym, start, end, path)

def run_dl(sym, start, end, path):
    a = download_tick_from_dukascopy
    while True:
        try:
            start_dl(a, sym, start, end, path)
            break
        except Exception as e:
            print(e)
            # get last date of dl, then continue
            d = get_all(sym)
            start = datetime.strftime(d.index[-1], "%Y-%m-%d")
            print('restart at', start)

In [3]:
if path not in os.listdir():
    os.mkdir(path)

# inst = 'USA500IDXUSD'
# inst = 'USATECHIDXUSD'
inst = 'BTCUSD'
if inst not in os.listdir(path):
    os.mkdir('{}/{}'.format(path, inst))

In [4]:
data = run_dl(
    inst, "2020-01-05", "2020-01-05", './{}/{}'.format(path, inst)
)

downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/00/05/00h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/00/05/01h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/00/05/02h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/00/05/03h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/00/05/04h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/00/05/05h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/00/05/06h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/00/05/07h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/00/05/08h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/00/05/09h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/00/05/10h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2020/

In [5]:
# inst = "USA500IDXUSD"
data = get_all(
    inst, './{}/{}/'.format(path, inst)
)

In [6]:
data

,bid,ask,bidv,askv
ts,,,,
2020-01-05 21:00:00,7482.0,7366.9,33.0,27.0
2020-01-05 21:01:00,7480.5,7365.5,33.0,27.0
2020-01-05 21:02:00,7470.5,7355.5,33.0,27.0
2020-01-05 21:03:00,7469.4,7354.3,33.0,27.0
2020-01-05 21:04:00,7469.3,7354.2,33.0,27.0
...,...,...,...,...
2020-01-05 23:55:00,7400.3,7285.9,25.0,7500.0
2020-01-05 23:56:00,7401.0,7286.7,25.0,7500.0
2020-01-05 23:57:00,7410.8,7296.4,25.0,7500.0


In [7]:
path = '../../Downloads/23h_ticks.bi5'

import lzma
import struct
import pandas as pd

def bi5_to_df(filename, fmt):
    chunk_size = struct.calcsize(fmt)
    data = []
    with lzma.open(filename) as f:
        while True:
            chunk = f.read(chunk_size)
            if chunk:
                data.append(struct.unpack(fmt, chunk))
            else:
                break
    df = pd.DataFrame(data)
    return df

In [8]:
bi5_to_df(path, '>3I2f')

,0,1,2,3,4
0,490,74034,72907,0.000025,0.0075
1,1219,74034,72908,0.000025,0.0075
2,2972,74039,72908,0.000025,0.0075
3,3916,74039,72910,0.000025,0.0075
4,8907,74039,72909,0.000025,0.0075
...,...,...,...,...,...
5048,3592259,74094,72949,0.000025,0.0075
5049,3593174,74094,72948,0.000025,0.0075
5050,3596909,74094,72947,0.000025,0.0075
5051,3598902,74094,72948,0.000025,0.0075


In [16]:
from urllib import request
from lzma import LZMADecompressor, LZMAError, FORMAT_AUTO

In [20]:
def decompress_lzma(data):
    results = []
    len(data)
    while True:
        decomp = LZMADecompressor(FORMAT_AUTO, None, None)
        try:
            res = decomp.decompress(data)
        except LZMAError:
            if results:
                break
            else:
                raise
        results.append(res)
        data = decomp.unused_data
        if not data:
            break
        if not decomp.eof:
            raise LZMAError("Compressed data ended before the end-of-stream marker was reached")
    return b"".join(results)
def normalize_tick(symbol, day, time, ask, bid, ask_vol, bid_vol):
    date = day + timedelta(milliseconds=time)

    # TODO 網羅する。この通過ペア以外も有るかも
    if any(map(lambda x: x in symbol.lower(), ['usdrub', 'xagusd', 'xauusd', 'jpy'])):
        point = 1000
    elif any(map(lambda x: x in symbol.upper(), ['USATECHIDXUSD', 'USA500IDXUSD'])):
        point = 1000
    elif any(map(lambda x: x in symbol.upper(), ['BTCUSD'])):
        point = 10
    else:
        point = 100000

    return [date, ask/point, bid/point, round(ask_vol * 1000000), round(bid_vol * 1000000)]

In [21]:
symbol = 'BTCUSD'
day = '2021-01-05'
day = datetime.strptime(day, '%Y-%m-%d')

url_prefix='https://datafeed.dukascopy.com/datafeed'
ticks_day = []
for h in range(0, 2):
    file_name = f'{h:02d}h_ticks.bi5'
    url = f'{url_prefix}/{symbol}/{day.year:04d}/{day.month-1:02d}/{day.day:02d}/{file_name}'
    print(f'downloading: {url}')
    req = request.Request(url)
    try:
        with request.urlopen(req) as res:
            res_body = res.read()
    except HTTPError:
        print('download failed. continuing..')
        continue
    if len(res_body):
        try:
            data = decompress_lzma(res_body)
        except LZMAError:
            print('decompress failed. continuing..')
            continue
    else:
        data = []
    tokenized_data = tokenize(data)
    ticks_hour = list(map(lambda x: normalize_tick(symbol, day + timedelta(hours=h), *x), tokenized_data))
    ticks_day.extend(ticks_hour)

downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2021/00/05/00h_ticks.bi5
downloading: https://datafeed.dukascopy.com/datafeed/BTCUSD/2021/00/05/01h_ticks.bi5


In [40]:
# res_body
# decompress_lzma(res_body)
struct.unpack('>3I2f', res_body)

error: unpack requires a buffer of 20 bytes

In [9]:
def tokenize(buffer):
    # fmt = '>3I2f'
    token_size = 20
    # token_size = struct.calcsize(fmt)
    token_count = int(len(buffer) / token_size)
    tokens = list(map(lambda x: struct.unpack_from('>3I2f', buffer, token_size * x), range(0, token_count)))
    return tokens

In [41]:
len(res_body)

54116

In [64]:
buffer = decompress_lzma(res_body)
token_count = int(len(buffer) / 20)
# list(map(
#     lambda x: struct.unpack_from(
#         '>3I2f', res_body, int(len(res_body) / 20) * x
#     ), 
#     range(0, len(res_body))
# ))
tokens = list(map(
    lambda x: struct.unpack_from('>3I2f', buffer, 20 * x), 
    range(0, token_count)
))

In [65]:
pd.DataFrame(tokens)

,0,1,2,3,4
0,487,324774,324425,0.000007,1.000000e-06
1,637,324767,324425,0.000007,1.000000e-06
2,747,324767,324445,0.000007,5.000000e-06
3,872,324768,324445,0.000007,5.000000e-06
4,1073,324768,324405,0.000007,1.100000e-05
...,...,...,...,...,...
13943,3598933,328394,327919,0.000007,2.200000e-05
13944,3599191,328408,327919,0.000007,2.200000e-05
13945,3599448,328372,327919,0.000007,2.200000e-05
13946,3599699,328370,327919,0.000007,2.200000e-05


In [57]:
int(len(res_body) / 20)

2705

In [58]:
range(0, len(res_body))

range(0, 54116)